In [38]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://host-192-168-1-153-ldsa:7077") \
        .appName("benblamey_hdf5")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","60s")\
        .config("spark.executor.cores",1)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [35]:
import h5py
import sys
import io

# h5py doesn't know anything about HDFS, and will be unable to work with the 'magic' HDFS filenames.
# As a workaround, we can load the file into memory with Spark and then read the file.

In [39]:
rdd = spark_context.binaryFiles("hdfs://host-192-168-1-153-ldsa:9000/millionsongs/data/A/A/A/TRAAAAW128F429D538.h5")
# or load all the files with some wildcards...
#rdd = spark_context.binaryFiles("hdfs://host-192-168-1-153-ldsa:9000/millionsongs/data/*/*/*/*")

def f(x):
    # x[0] = filename
    # x[1] = binary content
    with h5py.File(io.BytesIO(x[1])) as f:
        # drill down with a path
        # we need to convert to a "python list" -- custom types won't survive the round-trip via Java...
        return list(f.keys())
        
        g = f['metadata']['artist_terms']
        # g is a 'dataset' type.
        return g[1]

data = rdd.collect()

In [40]:
spark_context.stop()

In [65]:
def file_tuple_to_h5py(file):
    return h5py.File(io.BytesIO(file[0][1]))

dataset = file_tuple_to_h5py(data)



In [110]:
analysis = dataset['analysis']
analysis_attr = list(metadata)
metadata = dataset['metadata']
metadata_attr = list(metadata)
musicbrainz = dataset['musicbrainz']
musicbrainz_attr = list(musicbrainz)

print(list(analysis['songs']))
    

[(22050, b'a222795e07cd65b7a530f1346f520649', 0., 218.93179, 0.247, 0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.736, -11.197, 0, 0.636, 218.932, 92.198, 4, 0.778, b'TRAAAAW128F429D538')]
